<h1>EECS 549 Final Project</h1>

Data exploration & basic mainipulation

In [1]:
import os 
import pandas as pd
import json
import csv
from tqdm import tqdm
import numpy as np

In [2]:
posts = pd.read_csv("../data/ten-million-reddit-answers-posts.csv")

comments = pd.read_csv("../data/ten-million-reddit-answers-comments.csv")

<h2>Combine datasets</h2>

In [3]:
posts['created_date'] = pd.to_datetime(posts['created_utc'], unit = 's')

In [4]:
comments['post permalink'] = comments['permalink'].apply(lambda s: "/".join(s.split("/")[:-2]) + "/")

In [5]:
merge = posts.merge(comments, how = 'left', left_on = 'permalink', right_on = 'post permalink')

In [6]:
merge.shape

(9999980, 24)

In [7]:
merge.groupby('title')['body'].nunique().agg(['mean', 'median', 'min', 'max'])

mean         15.300088
median        4.000000
min           0.000000
max       51673.000000
Name: body, dtype: float64

In [8]:
posts['created_date'] = pd.to_datetime(posts['created_utc'], unit = 's')

In [9]:
posts['created_date'].agg(['min', 'max'])

min   2010-02-18 22:44:06
max   2020-11-30 23:59:39
Name: created_date, dtype: datetime64[ns]

<h2>Create new dataset made up of full posts</h2>

In [10]:
id_map = pd.read_csv("../data/id_map.csv", header = None)
id_map.columns = ['new', 'old']

In [11]:
merge = merge.drop(columns = ['type_y', 
                              'id_y', 
                              'subreddit.id_y', 
                              'subreddit.name_y', 
                              'subreddit.nsfw_y',
                              'created_utc_y', 
                              'permalink_y', 'score_y'])
           

In [12]:
merge.columns = [col[:-2] if '_x' in col else col for col in merge.columns]

In [13]:
merge = merge.sort_values(by = 'created_utc', ascending = True)

In [14]:
merge = merge.merge(id_map, left_on = 'id', right_on = 'old', how = 'left')
merge = merge.drop(columns = 'old')
merge = merge.rename(columns = {'new':'docid'})

In [15]:
def fill_idx(s, df = merge):
    if pd.isnull(s) == True:
        max = df['docid'].max()
        return (max + 1)
    return s

In [ ]:
merged['docid'] = merge['docid'].apply(fill_idx)

In [13]:
ids = np.random.choice(list(set(merge['id'].values)), 100000, replace = False)

In [14]:
merge = merge[merge['id'].isin(ids)]

In [15]:
merge.to_csv("../data/ask_reddit.csv", index = False)

In [16]:
titles = list(merge['title'].astype(str).values)
comments = list(merge['body'].astype(str).values)
idx = list(merge['id'].values)

In [23]:
idx_to_post = {}
docid_to_postid = {}

docid = 0
for i in tqdm(range(len(idx))):
    id = idx[i]
    title = titles[i]
    comment = comments[i]
    if '[deleted]' in comment or '[removed]' in comment or '**PLEASE READ THIS MESSAGE IN ITS ENTIRETY BEFORE TAKING ACTION.**' in comment:
        continue
    if id not in list(docid_to_postid.values()):
        docid += 1
        idx_to_post[docid] = title + " " + comment + " "
        docid_to_postid[docid] = id
        continue
    curr = idx_to_post[docid]
    if len(curr) <= 25000:
        idx_to_post[docid] = idx_to_post[docid] + comment + " "

 53%|██████████████████████████████████████████▍                                     | 885635/1668782 [05:44<05:04, 2573.88it/s]


KeyboardInterrupt: 

In [18]:
list(idx_to_post.keys()) == list(docid_to_postid.keys())

True

In [19]:
len(idx), len(set(idx))

(1668782, 100000)

<h2>Write dataset to file</h2>

In [20]:
with open("../data/ask_reddit_posts.jsonl", "w") as f:
    for k, v in list(idx_to_post.items()):
        d = {'docid': k, 'text': v}
        print(json.dumps(d), file = f)

In [22]:
with open("../data/id_map.csv", "w") as f: 
    writer = csv.writer(f)
    writer.writerows(docid_to_postid.items())